# Expert System with Shell

In the cells below:

1. Write in the KB that we converged upon in the previous activity
2. Determine how the ask predicate works, and be prepared to explain after the breakout
3. Determine how the Python code using the PySWIP interface works, and be prepared to explain this during the debrief.

In [7]:
KB = """
% Enter your KB below this line:

problem(battery) :- not(engine(turn_over)), battery(bad).
battery(bad) :- lights(weak).
battery(bad) :- radio(weak).
problem(flooded) :- engine(turn_over), smell(gas).
problem(out_of_gas) :- gas_gauge(empty).

engine(X) :- not(check_if_care(engine, X)).
lights(X) :- ask(lights,X).
% engine(X) :- ask(engine,X).
radio(X) :- ask(radio,X).
smell(X) :- ask(smell,X).
gas_gauge(X) :- ask(gas_gauge,X).

check_if_care(A, X) :- care(A), not(ask(A, X)).
care(A) :- atom_concat(care_check_, A, X), ask(X, '').

% The code below implements the prompting to ask the user:


% Asking clauses

multivalued(none).

ask(A, V):-
known(y, A, V), % succeed if true
!.	% stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

ask(A, V):-
not(multivalued(A)),
write_py(A:not_multivalued),
known(y, A, V2),
V \== V2,
!, fail.

ask(A, V):-
read_py(A,V,Y), % get the answer
asserta(known(Y, A, V)), % remember it
Y == y.	% succeed or fail
"""

with open("KB_A.pl", "w") as text_file:
    text_file.write(KB)

In [9]:
# The code here will ask the user for input based on the askables
# It will check if the answer is known first


from pyswip.prolog import Prolog
from pyswip.easy import *

prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    Y.unify(raw_input(str(A) + " is " + str(V) + "?"))
    return True


write_py.arity = 1
read_py.arity = 3

registerForeign(read_py)
registerForeign(write_py)

prolog.consult("KB_A.pl") # open the KB
call(retractall(known))
for soln in prolog.query("problem(X).", maxresult=1):
    print("Your problem is " + soln['X'])

:(care_check_engine, not_multivalued)
care_check_engine is ?n
:(smell, not_multivalued)
smell is gas?y
Your problem is flooded


# Flight Booking System

The code in the cells below implements a simple command parser utilizing DCGs for a flight booking system.  Your task is to understand how it works by running it and making modifications to evaluate the effect.

1. Determine how the Prolg code works.  In particular, focus on the following:
    * Study the DCGs - they are more complicated than what we have seen so far (they have arguments!!).  Try to follow the grammar being defined by them.  The grammar is related to processing commands.  Some example commands have been shown in the output so you can see how the parser works.  Work out what the semicolons in the operation(exit) DCG do, as well as the curly brackets in the argument(FLIGHT) DCG.
    * run_command is the workhorse predicate that actually invokes the command parser.  See if you can ascertain the purpose of the "univ" operator, denoted by "=..". Also, the command top-level DCG is actually called with three parameters rather than two as in a normal difference list input.  See if you can work out where the extra argument comes from.
    * Why is the "fail" predicate used at the end of each of the report predicates?
    * This command parser is very flimsy - it does minimal input checking.  Discuss how we might make it more robust.

In [3]:
# Generate a random filename for the KB if one does not exist already

import random

if not 'fileName' in locals():
    fileName = 'KB_' + str(random.getrandbits(128)) + '.pl'

KB = """
run_command(TOKENS, COMMAND) :-
   command(CLIST, TOKENS, []),
   COMMAND =.. CLIST,
   call(COMMAND).

% Define the exit predicate to do nothing.
exit.

%%% DCGs

command([OP|ARGS]) --> operation(OP), arguments(ARGS).

arguments([ARG|ARGS]) --> argument(ARG), arguments(ARGS).
arguments([]) --> [].

operation(report) --> [list].
operation(book) --> [book].
operation(cancel) --> [cancel].
operation(exit) --> ([exit]; [quit]; [bye]).

argument(passengers) --> [passengers].
argument(flights) --> [flights].

argument(FLIGHT) --> [FLIGHT], {flight(FLIGHT)}.
argument(PASSENGER) --> [PASSENGER].

% Flights

flight(aa101).
flight(sq238).
flight(mi436).
flight(oz521).

% Command predicates

report(flights) :-
   flight(F),
   write_py(F),
   fail.
report(_).

report(passengers, FLIGHT) :-
   booked(PASSENGER, FLIGHT),
   write_py(PASSENGER),
   fail.
report(_, _).

book(PASSENGER, FLIGHT) :-
   assertz(booked(PASSENGER, FLIGHT)).

cancel(PASSENGER, FLIGHT) :-
   retract(booked(PASSENGER, FLIGHT)).

%%%%% End Prolog File
"""

# Write out the KB to file here
with open(fileName, "w") as text_file:
    text_file.write(KB)

In [7]:
# The code here will ask the user for input based on the askables
# It will check if the answer is known first
from pyswip.prolog import Prolog
from pyswip.easy import *

prolog = Prolog() # Global handle to interpreter

# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

write_py.arity = 1

registerForeign(write_py)

prolog.consult(fileName) # open the KB


# Main command loop:

prolog.consult(fileName)
print("%%%% Minerva flight booking system %%%%")
sys.stdout.flush()

running = True

while running:
    CSTR = "[" + ", ".join(input("Enter command: ").split(" ")) + "]" # Tokenize
    for soln in prolog.query("run_command(%s,COMMAND)." % CSTR, maxresult=1):
        if soln['COMMAND'] == "exit":
            print("Goodbye.")
            running = False


%%%% Minerva flight booking system %%%%
Enter command: list flights
aa101
sq238
mi436
oz521
Enter command: exit
Goodbye.
